In [1]:
from threading import Thread

In [2]:
class Worker(Thread):
    'computes x, v and a of the ith body'
    def __init__(self, *args, **kwargs):
        super(Worker, self).__init__(*args, **kwargs)
        self.inputs =[]
        self.results = []
        self.running = True
        self.daemon = True
        self.start()

    def run(self):
        while self.running:
            if len(self.inputs) == 0:
                continue
            i, x0, v0, G, m, dt = self.inputs.pop(0)
            a_i0 = a(i, x0, G, m)
            v_i1 = a_i0 * dt + v0[i]
            x_i1 = a_i0 * dt ** 2 + v0[i] * dt + x0[i]
            result = (i, x_i1, v_i1)
            self.results.append(result)

In [ ]:
def remove_i(x, i):
    shape = (x.shape[0]-1,) + x.shape[1:]
    y = np.empty(shape, dtype=float)
    y[:i] = x[:i]
    y[i:] = x[i+1:]
    return y

def a(i, x, G, m):
    'the accerleration of the ith mass'
    x_i = x[i]
    x_j = remove_i(x, i)
    m_j = remove_i(m, i)
    diff = x_j - x_i
    mag3 = np.sum(diff**2, axis=1)**1.5
    result = G * np.sum(diff * (m_j / mag3)[:,np.newaxis], axis=0)
    return result
    
def initial_cond(N, D):
    x0 = np.random.rand(N, D)
    v0 = np.zeros((N,D), dtype=float)
    m = np.ones(N, dtype=float)
    return x0, v0, m
        